In [1]:
import pandas as pd
import numpy as np
import os 

In [2]:
dataset_list = ['MUTAG','PTC_MR','ENZYMES','PROTEINS','DD']
method_list = ['HKS','WKS']
column_list = ['dataset','method','C','gamma','w','sinkhorn','acc','std']


In [3]:
data = []
for dataset in dataset_list:
    for method in method_list:
        Path = './results/'+dataset+'/'+method+'/'
        list_of_files = sorted( filter( lambda x: os.path.isfile(os.path.join(Path, x)),
                                os.listdir(Path) ) )
        result_files = [pd.read_csv(Path+i) for i in list_of_files]
        means = np.array([np.mean(file["accuracy"]) for file in result_files])
        stds = np.array([np.std(file["accuracy"]) for file in result_files])
        gs = [np.mean(file["gamma"]) for file in result_files]
        Cs = [np.mean(file["C"]) for file in result_files]
        ws = [np.mean(file["w"]) for file in result_files]
        # best_df = result_files[np.argmax(means)].iloc[1]
        for i in range(3):
            data_unit = [dataset,method,Cs[i],gs[i],ws[i],False,means[i],stds[i]]
            if dataset == 'DD':
                data_unit[-3] = True
            data.append(data_unit)

In [4]:
df = pd.DataFrame(data,columns=column_list)

In [5]:
df

,dataset,method,C,gamma,w,sinkhorn,acc,std
0,MUTAG,HKS,10.0,1.00,0.00,False,0.786842,0.059213
1,MUTAG,HKS,10.0,1.00,0.30,False,0.898830,0.043922
2,MUTAG,HKS,10.0,1.00,1.00,False,0.803509,0.081377
3,MUTAG,WKS,1000.0,0.10,0.00,False,0.808480,0.053777
4,MUTAG,WKS,1000.0,0.10,0.60,False,0.925146,0.036030
5,MUTAG,WKS,1000.0,0.10,1.00,False,0.851170,0.057602
6,PTC_MR,HKS,1000.0,1.00,0.00,False,0.570000,0.070525
7,PTC_MR,HKS,1000.0,1.00,0.35,False,0.619244,0.045103
8,PTC_MR,HKS,1000.0,1.00,1.00,False,0.587143,0.052070
9,PTC_MR,WKS,1.0,10.00,0.00,False,0.595798,0.040925


In [6]:
df.to_csv('ablation.csv')

In [16]:
temp_df = df[df['w'] == 1.00]
hks_acc = [acc for acc in temp_df[temp_df['method']=='HKS']['acc']]
hks_std = [std for std in temp_df[temp_df['method']=='HKS']['std']]
hks_res = ['{:2.2f}+-{:2.2f}%'.format(hks_acc[i]*100,(hks_std[i]/np.sqrt(10))*100) for i in range(len(hks_acc))]
hks_df = pd.DataFrame([hks_acc,hks_std,hks_res],columns=dataset_list)

In [17]:
hks_df

,MUTAG,PTC_MR,ENZYMES,PROTEINS,DD
0,0.85117,0.633613,0.235,0.722104,0.74778
1,0.057602,0.06022,0.028333,0.024451,0.035377
2,85.12+-1.82%,63.36+-1.90%,23.50+-0.90%,72.21+-0.77%,74.78+-1.12%
